<h2> Importing libraries

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

#Importing folium
import folium
from folium.plugins import HeatMap

import warnings
warnings.filterwarnings('ignore')

<h2> Importing dataset

In [17]:
df = pd.read_csv("dataset//Global_terrorist.csv", encoding="ISO-8859-1")

In [18]:
df

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181686,201712310022,2017,12,31,NaN,0,NaN,182,Somalia,11,...,NaN,"""Somalia: Al-Shabaab Militants Attack Army Che...","""Highlights: Somalia Daily Media Highlights 2 ...","""Highlights: Somalia Daily Media Highlights 1 ...",START Primary Collection,0,0,0,0,NaN
181687,201712310029,2017,12,31,NaN,0,NaN,200,Syria,10,...,NaN,"""Putin's 'victory' in Syria has turned into a ...","""Two Russian soldiers killed at Hmeymim base i...","""Two Russian servicemen killed in Syria mortar...",START Primary Collection,-9,-9,1,1,NaN
181688,201712310030,2017,12,31,NaN,0,NaN,160,Philippines,5,...,NaN,"""Maguindanao clashes trap tribe members,"" Phil...",NaN,NaN,START Primary Collection,0,0,0,0,NaN
181689,201712310031,2017,12,31,NaN,0,NaN,92,India,6,...,NaN,"""Trader escapes grenade attack in Imphal,"" Bus...",NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN


<h2> Insights

In [22]:
#Insight with folium example by prof
filtered_df = df[df['region_txt'] == 'Middle East & North Africa']

# Filter by year (e.g., incidents after 2000)
filtered_df = filtered_df[filtered_df['iyear'] >= 2000]

# Filter by attack type (e.g., bombings)
filtered_df = filtered_df[filtered_df['attacktype1_txt'] == 'Bombing/Explosion']

# Filter by significant impact (e.g., incidents with at least 10 fatalities)
filtered_df = filtered_df[filtered_df['nkill'] >= 10]

print(f"Filtered dataset size: {len(filtered_df)} rows")

# Aggregate incidents by latitude and longitude
aggregated_df = filtered_df.groupby(['latitude', 'longitude']).size().reset_index(name='incident_count')

print(aggregated_df.head())

# Aggregate incidents by location
aggregated_df = filtered_df.groupby(['latitude', 'longitude']).size().reset_index(name='incident_count')

# Normalize weights (optional)
aggregated_df['normalized_weight'] = (aggregated_df['incident_count'] - aggregated_df['incident_count'].min()) / \
                                (aggregated_df['incident_count'].max() - aggregated_df['incident_count'].min())

# Prepare heat_data
heat_data = aggregated_df[['latitude', 'longitude', 'normalized_weight']].values.tolist()

# Create a base map
m = folium.Map(location=[30, 40], zoom_start=4)

# Add heatmap layer
HeatMap(heat_data).add_to(m)

# Save or display the map
m.save('terrorism_heatmap.html')
m

Filtered dataset size: 1737 rows
    latitude  longitude  incident_count
0  12.743795  44.843678               1
1  12.760069  44.890266               1
2  12.825647  44.846416              12
3  12.832581  45.027071               1
4  12.867085  44.981911               6


<h2> Regional Insight with folium mapping

<h3> Which areas in North America have experienced the highest concentration of armed assaults since 2000?

In [73]:
filtered_df = df[
    (df['region_txt'] == 'North America') &
    (df['iyear'] >= 2000) &
    (df['attacktype1_txt'] == 'Armed Assault')
]

aggregated_df = filtered_df.groupby(['latitude', 'longitude']).size().reset_index(name='incident_count')
aggregated_df['normalized_weight'] = (
    (aggregated_df['incident_count'] - aggregated_df['incident_count'].min()) /
    (aggregated_df['incident_count'].max() - aggregated_df['incident_count'].min())
)

heat_data = aggregated_df[['latitude', 'longitude', 'normalized_weight']].values.tolist()
regional_heatmap = folium.Map(location=[40, -100], zoom_start=4, tiles='cartodbpositron')
HeatMap(heat_data, radius=10, blur=15, max_zoom=6).add_to(regional_heatmap)
regional_heatmap.save('regional_heatmap.html')
regional_heatmap


<h2> Where do the most common types of attacks occur globally?

In [74]:
attack_counts = df['attacktype1_txt'].value_counts()
most_common_attack = attack_counts.idxmax()
print(f"Most common attack type: {most_common_attack}")

filtered_df = df[df['attacktype1_txt'] == most_common_attack]

region_counts = filtered_df['region_txt'].value_counts()
top_region = region_counts.idxmax()
print(f"Region with the highest occurrences of '{most_common_attack}': {top_region}")

top_region_df = filtered_df[filtered_df['region_txt'] == top_region]

aggregated_df = top_region_df.groupby(['latitude', 'longitude']).size().reset_index(name='incident_count')
aggregated_df['normalized_weight'] = (
    (aggregated_df['incident_count'] - aggregated_df['incident_count'].min()) /
    (aggregated_df['incident_count'].max() - aggregated_df['incident_count'].min())
)

heat_data = aggregated_df[['latitude', 'longitude', 'normalized_weight']].dropna().values.tolist()
region_map = folium.Map(location=[30, 70], zoom_start=4, tiles='cartodbpositron')
HeatMap(heat_data, radius=10, blur=15, max_zoom=6).add_to(region_map)
region_map.save('most_common_attack_region_heatmap.html')
region_map


Most common attack type: Bombing/Explosion
Region with the highest occurrences of 'Bombing/Explosion': Middle East & North Africa


<h2> Crime Insight with Foilum Heatmap
    

In [75]:
filtered_df = df[df['iyear'] >= 2000]
attack_type = 'Bombing/Explosion'
filtered_df = filtered_df[filtered_df['attacktype1_txt'] == attack_type]

aggregated_df = filtered_df.groupby(['latitude', 'longitude']).size().reset_index(name='incident_count')
aggregated_df['normalized_weight'] = (
    (aggregated_df['incident_count'] - aggregated_df['incident_count'].min()) /
    (aggregated_df['incident_count'].max() - aggregated_df['incident_count'].min())
)

heat_data = aggregated_df[['latitude', 'longitude', 'normalized_weight']].dropna().values.tolist()
crime_map = folium.Map(location=[30, 70], zoom_start=4, tiles='cartodbpositron')
HeatMap(heat_data, radius=10, blur=15, max_zoom=6).add_to(crime_map)

crime_map.save('crime_type_heatmap.html')
crime_map

<h3> Where do the most deadly Bombing/Explosion incidents (with at least 10 fatalities) occur globally?

In [48]:
filtered_df = df[(df['attacktype1_txt'] == 'Bombing/Explosion') & (df['nkill'] >= 10)]

aggregated_df = filtered_df.groupby(['latitude', 'longitude']).size().reset_index(name='incident_count')
aggregated_df['normalized_weight'] = (
    (aggregated_df['incident_count'] - aggregated_df['incident_count'].min()) /
    (aggregated_df['incident_count'].max() - aggregated_df['incident_count'].min())
)

heat_data = aggregated_df[['latitude', 'longitude', 'normalized_weight']].dropna().values.tolist()
crime_map_1 = folium.Map(location=[30, 70], zoom_start=4, tiles='cartodbpositron')
HeatMap(heat_data, radius=10, blur=15, max_zoom=6).add_to(crime_map_1)
crime_map_1.save('crime_insight_heatmap_nkill_10.html')
crime_map_1


<h2> Casualty Analysis with heatmap

<h3> What are the global hotspots for incidents with significant casualties (fatalities or injuries)?

In [52]:
filtered_df = df[(df['nkill'] >= 10) & (df['nwound'] >= 10)]

aggregated_df = filtered_df.groupby(['latitude', 'longitude']).size().reset_index(name='incident_count')
aggregated_df['normalized_weight'] = (
    (aggregated_df['incident_count'] - aggregated_df['incident_count'].min()) /
    (aggregated_df['incident_count'].max() - aggregated_df['incident_count'].min())
)

heat_data = aggregated_df[['latitude', 'longitude', 'normalized_weight']].dropna().values.tolist()
casualty_map = folium.Map(location=[30, 70], zoom_start=4, tiles='cartodbpositron')
HeatMap(heat_data, radius=10, blur=15, max_zoom=6).add_to(casualty_map)
casualty_map.save('casualty_analysis_heatmap.html')
casualty_map


<h3> Where do incidents with significant casualties from armed assault (10+ fatalities/injuries) occur

In [57]:
casualty_df = df[(df['attacktype1_txt'] == 'Armed Assault') & ((df['nkill'] >= 10) & (df['nwound'] >= 10))]

aggregated_df = casualty_df.groupby(['latitude', 'longitude']).size().reset_index(name='incident_count')
aggregated_df['normalized_weight'] = (
    (aggregated_df['incident_count'] - aggregated_df['incident_count'].min()) /
    (aggregated_df['incident_count'].max() - aggregated_df['incident_count'].min())
)

heat_data = aggregated_df[['latitude', 'longitude', 'normalized_weight']].dropna().values.tolist()
attack_map = folium.Map(location=[30, 70], zoom_start=4, tiles='cartodbpositron')
HeatMap(heat_data, radius=10, blur=15, max_zoom=6).add_to(attack_map)
attack_map.save('attack_type_casualty_analysis_heatmap.html')
attack_map


<h2> Demographic Analysis

<h3> Where are the most terrorist incidents targeting Private Citizens & Property occurring?

In [61]:
target_df = df[df['targtype1_txt'] == 'Private Citizens & Property']

aggregated_df = target_df.groupby(['latitude', 'longitude']).size().reset_index(name='incident_count')
aggregated_df['normalized_weight'] = (
    (aggregated_df['incident_count'] - aggregated_df['incident_count'].min()) /
    (aggregated_df['incident_count'].max() - aggregated_df['incident_count'].min())
)

heat_data = aggregated_df[['latitude', 'longitude', 'normalized_weight']].dropna().values.tolist()
target_map = folium.Map(location=[30, 70], zoom_start=4, tiles='cartodbpositron')
HeatMap(heat_data, radius=10, blur=15, max_zoom=6).add_to(target_map)
target_map.save('target_type_demographic_analysis_heatmap.html')
target_map


<h3> Where are the most terrorist incidents by Black Nationalists occurring?

In [66]:
group_df = df[df['gname'] == 'Black Nationalists']
aggregated_df = group_df.groupby(['latitude', 'longitude']).size().reset_index(name='incident_count')
aggregated_df['normalized_weight'] = (
    (aggregated_df['incident_count'] - aggregated_df['incident_count'].min()) /
    (aggregated_df['incident_count'].max() - aggregated_df['incident_count'].min())
)

heat_data = aggregated_df[['latitude', 'longitude', 'normalized_weight']].dropna().values.tolist()
group_map = folium.Map(location=[30, 70], zoom_start=4, tiles='cartodbpositron')
HeatMap(heat_data, radius=10, blur=15, max_zoom=6).add_to(group_map)
group_map.save('attacker_group_demographic_analysis_heatmap.html')
group_map


<h2> Target Analysis with Heatmap

<h3> Where are the most terrorist incidents targeting military personnel occurring?

In [69]:
target_military = df[df['targtype1_txt'] == 'Military']

aggregated_df = target_military.groupby(['latitude', 'longitude']).size().reset_index(name='incident_count')
aggregated_df['normalized_weight'] = (
    (aggregated_df['incident_count'] - aggregated_df['incident_count'].min()) /
    (aggregated_df['incident_count'].max() - aggregated_df['incident_count'].min())
)

heat_data = aggregated_df[['latitude', 'longitude', 'normalized_weight']].dropna().values.tolist()
target_map = folium.Map(location=[30, 70], zoom_start=4, tiles='cartodbpositron')
HeatMap(heat_data, radius=10, blur=15, max_zoom=6).add_to(target_map)

target_map.save('two_target_analysis_heatmap.html')
target_map 


<h3> Which areas have the highest number of terrorist incidents targeting government institutions?

In [71]:
gov_df = df[df['targtype1_txt'] == 'Government (General)']

aggregated_df = gov_df.groupby(['latitude', 'longitude']).size().reset_index(name='incident_count')
aggregated_df['normalized_weight'] = (
    (aggregated_df['incident_count'] - aggregated_df['incident_count'].min()) /
    (aggregated_df['incident_count'].max() - aggregated_df['incident_count'].min())
)

heat_data = aggregated_df[['latitude', 'longitude', 'normalized_weight']].dropna().values.tolist()
gov_map = folium.Map(location=[20, 30], zoom_start=3, tiles='cartodbpositron')
HeatMap(heat_data, radius=10, blur=15, max_zoom=6).add_to(gov_map)

gov_map.save('single_target_government_heatmap.html')
gov_map